In [2]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

In [3]:
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
ruta

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [5]:
# funcon para ejecutar una script SQL
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

## USUARIOS ACTIVOS

In [7]:
def get_activos():
    query_1 = 'select DATE(tok.created_at) as created_at, usu.codigo, usu.name, usu.apellido_paterno, usu.apellido_materno, usu.username, usu.tipo, \'reg acces\' as origen\
                        from oauth_access_tokens tok\
                        inner join users usu on tok.user_id=usu.id\
                        WHERE tok.created_at >= DATE_SUB(NOW(), INTERVAL 6 MONTH) and usu.tipo=\'propietario\''
    usuarios_acces = dbQuery(query_1)
    data_acces = pd.DataFrame(usuarios_acces)
    
    query_2 = 'select DATE(ac.created_at) as created_at, us.codigo, us.name, us.apellido_paterno, us.apellido_materno, us.username, us.tipo, \'reg. labor\'  as origen\
                from actividades ac\
                inner join users us on us.id = ac.usuario_registro\
                WHERE ac.created_at >= DATE_SUB(NOW(), INTERVAL 6 MONTH)'
    usuarios_reg = dbQuery(query_2)
    data_reg = pd.DataFrame(usuarios_reg)
    data = pd.concat([data_acces, data_reg], ignore_index=True)
    data.to_excel(ruta + '/_agrocittca-data/' + "usuarios_activos.xlsx", index=False)

def get_variedades():
    query = 'select * from variedad'
    variedades = dbQuery(query)
    data = pd.DataFrame(variedades)
    data.to_excel(ruta + '/_agrocittca-data/' + "variedades.xlsx", index=False)
get_variedades()

In [8]:
periodos = {16, 17}
periodos = "(" + ", ".join(map(str, periodos)) + ")"
periodos

'(16, 17)'

In [9]:
props = {500}
props = "(" + ", ".join(map(str, props)) + ")"
props

'(500)'

In [25]:
def get_propiedades():
    query = f"select * from propiedad\
                where estado = \'activo\'\
                and periodo_id in {periodos}\
                and codigo in {props}"
    propiedades = dbQuery(query)
    data = pd.DataFrame(propiedades)
    data.to_excel(ruta + '/_agrocittca-data/' + "propiedades.xlsx", index=False)
    return data

def get_periodos(peris):
    query = f"select * from periodo\
            where estado = 'activo'\
            and id in {peris}"
    periodos = dbQuery(query)
    data = pd.DataFrame(periodos)
    data.to_excel(ruta + '/_agrocittca-data/' + "periodos.xlsx", index=False)
    return data

def get_canheros(ca):
    query = f"select * from users\
            where estado = 'activo'\
            and id in {ca}"
    canheros = dbQuery(query)
    data = pd.DataFrame(canheros)
    data.to_excel(ruta + '/_agrocittca-data/' + "canheros.xlsx", index=False)
    return data

def get_lotes(props_ids):
    query = f"select distinct(l.id), l.* from propiedad prop\
                inner join lote l on l.propiedad_id = prop.id\
                inner join actividades ac on ac.lote_id = l.id\
                where l.estado = 'activo'\
                and ac.estado = 'activo'\
                and prop.id in {props_ids}"
    lotes = dbQuery(query)
    data = pd.DataFrame(lotes)
    #data['superficie'] = data['superficie'].str.replace('.', ',')
    data['superficie'] = data['superficie'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "lotes.xlsx", index=False)
    return data

def get_actividades(props_ids):
    query = f"select ac.* from propiedad prop\
                inner join lote l on l.propiedad_id = prop.id\
                inner join actividades ac on ac.lote_id = l.id\
                where l.estado = 'activo'\
                and ac.estado = 'activo'\
                and prop.id in {props_ids}"
    actividades = dbQuery(query)
    data = pd.DataFrame(actividades)
    data.to_excel(ruta + '/_agrocittca-data/' + "actividades.xlsx", index=False)
    return data

def get_tipo_actividad(periodos_ids):
    query = f"select * from tipo_actividad\
                where estado='activo'\
                and periodo_id in {periodos_ids}"
    tipo_ac = dbQuery(query)
    data = pd.DataFrame(tipo_ac)
    data.to_excel(ruta + '/_agrocittca-data/' + "tipo_actividad.xlsx", index=False)
    return data

def get_labor():
    query = f"select * from labor where estado='activo'"
    labor = dbQuery(query)
    data = pd.DataFrame(labor)
    data.to_excel(ruta + '/_agrocittca-data/' + "labor.xlsx", index=False)
    return data

def get_detalle_actividad(periodos_ids):
    query = f"select det_ac.* from detalle_actividad det_ac\
            inner join actividades ac on ac.id = det_ac.actividad_id\
            where det_ac.estado='activo'\
            and ac.periodo_id in {periodos_ids}"
    det_ac = dbQuery(query)
    data = pd.DataFrame(det_ac)
    data['cantidad'] = data['cantidad'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "detalle_actividad.xlsx", index=False)
    return data

def get_producto_act():
    query = f"select * from producto_actividad where estado = 'activo'"
    producto_act = dbQuery(query)
    data = pd.DataFrame(producto_act)
    data['precio_unitario'] = data['precio_unitario'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "producto_actividad.xlsx", index=False)
    return data

In [11]:
propiedades = get_propiedades()

In [12]:
periodos = set(propiedades['periodo_id'])
periodos = "(" + ", ".join(map(str, periodos)) + ")"
get_periodos(periodos)
periodos

'(16, 17)'

In [13]:
canheros = set(propiedades['usuario_id'])
canheros = "(" + ", ".join(map(str, canheros)) + ")"
get_canheros(canheros)
canheros

'(51)'

In [16]:
props_ids

'(3342, 3551)'

In [15]:
props_ids = set(propiedades['id'])
props_ids = "(" + ", ".join(map(str, props_ids)) + ")"
lotes = get_lotes(props_ids)
lotes

,id,l.id,codigo,nombre,slug,superficie,fecha_siembra,corte,estado_lote,rendimiento,estado,observacion,propiedad_id,usuario_id,variedad_id,textura_id,created_at,updated_at
0,28630,28630,00005000000L5,L5,l5-28630,45.22,0,122,4,53.44145750229,activo,,3342,17691,16,2,2024-01-31 08:11:02,2024-01-31 08:11:02


In [26]:
actividades = get_actividades(props_ids)
actividades

,id,superficie,slug,hora_inicio,hora_final,combustible,cantidad_combustible,estado,comentario,fecha_ejecucion,...,periodo_id,created_at,updated_at,mi_precio,tipo_actividad_mi_precio_id,precio_tipo_actividad,detalle_combustible_id,mi_precio_hora_tipo_actividad,tipo_actividad_mi_precio_hora_id,mi_precio_hora_tipo_actividad_valor
0,4530,45.22,actividad-4530,1,2,None,None,activo,None,2024-09-02,...,16,2024-10-06 22:25:36,2024-10-06 22:25:36,no,None,30,None,no,None,None


In [18]:
tipo_actividad = get_tipo_actividad(periodos)
tipo_actividad.head(3)

,id,nombre,fecha_inicio,fecha_fin,intervalo,slug,estado,precio_unitario,labor_id,periodo_id,created_at,updated_at,precio_hora
0,135,Rastra pesada (siembra),2024-03-01,2024-05-30,10,rastra-pesada-siembra-135,activo,35,1,16,2024-01-31 10:57:33,2024-01-31 10:57:33,0
1,136,Rastra liviana (siembra),2024-03-01,2024-05-30,20,rastra-liviana-siembra-136,activo,20,1,16,2024-01-31 10:57:33,2024-01-31 10:57:33,0
2,137,Surcado (siembra),2024-03-01,2024-05-30,5,surcado-siembra-137,activo,40,1,16,2024-01-31 10:57:33,2024-01-31 10:57:33,0


In [19]:
labor = get_labor()
labor

,id,nombre,slug,estado,foto,created_at,updated_at
0,1,Siembra,siembra-1,activo,/storage/images/labor/1vOIq0OHUkp5688Cv659WUt6...,2022-08-31 17:07:16,2023-02-06 10:08:38
1,2,Labores culturales caña hoja,labores-cultura-cana-hoja-2,activo,/storage/images/labor/zgSTvL1uPamNvnKP4Cud0L8W...,2022-08-31 17:07:16,2023-02-06 10:07:15
2,3,Cosecha,cosecha-3,activo,/storage/images/labor/8NxQPgUGsULjZjdoMwDGG18T...,2022-08-31 17:07:16,2023-02-06 10:08:12
3,4,Labores culturales caña soca,labores-cultura-cana-soca-4,activo,/storage/images/labor/10thlzt9POAScuPpNsZMPzK0...,2022-08-31 17:07:16,2024-04-01 13:54:15
4,5,Tutorial Uso de AGROCITTCA,tutorial-uso-de-agrocittca-5,activo,/storage/images/labor/oRBJnRvVhpvWRyXuBwqORaQN...,2022-08-31 17:07:16,2024-04-01 14:28:51
5,6,Gestión y Administración,gestion-y-administracion-6,activo,/imagen/foto_labor.png,2024-04-01 13:55:28,2024-04-01 13:55:28


In [28]:
detalle_ac = get_detalle_actividad(periodos)
detalle_ac.head(3)

,id,nombre_producto,dosis_recomendada,unidad_medida,cantidad,estado,actividad_id,producto_id,created_at,updated_at,codigo_grupo_producto,mi_precio,producto_act_miprecio_id,precio_producto
0,6475,UREA,0,KGS,196.0,activo,4513,24,2024-10-06 20:36:52,2024-10-06 20:36:52,c-04513,no,None,0.41
1,6476,UREA,0,KGS,1222.0,activo,4516,24,2024-10-06 21:12:06,2024-10-06 21:12:06,c-04516,no,None,0.41
2,6477,Cloruro de Potacio,0,KGS,611.0,activo,4516,189,2024-10-06 21:12:06,2024-10-06 21:12:06,c-04516,no,None,0.71


In [24]:
producto_act = get_producto_act()
producto_act.head(3)

,id,nombre,descripcion,precio_unitario,ingrediente_activo,dosis_recomendada,estado,slug,unidad_medida_id,categoria_producto_id,created_at,updated_at
0,1,Sunami,,166.76,FIPRONIL 80%,4,activo,sunami-1,1,1,2022-08-31 17:07:16,2022-08-31 17:07:16
1,3,Merlin Total,U.C.G.,147.50,Merlin Total SC600,0,activo,merlin-total-3,2,3,2022-08-31 17:07:16,2023-02-22 15:48:37
2,4,Flutolex,None,134.00,ISOXAFLUTOLE,0,activo,flutolex-4,2,3,2022-08-31 17:07:16,2023-01-30 09:44:36


In [34]:
get_activos()
get_actividades()

In [36]:
get_tipo_actividad()

In [38]:
get_labor()

In [42]:
get_maquinaria()

In [44]:
get_operario()

In [46]:
get_categoria_producto()

In [4]:
# SELECCIONAR TODOS LOS USUARIOS CAÑEROS ACTIVOS
#select codigo, name, apellido_paterno, apellido_materno, username from users where estado='activo' and tipo = 'propietario
cantidad_usuarios = dbQuery('select count(*) cantidad_usuarios from users where estado=\'activo\' and tipo = \'propietario\'')[0]['cantidad_usuarios']
cantidad_props = dbQuery('select count(*) cantidad_props from propiedad prop\
                            inner join periodo per on prop.periodo_id=per.id\
                            where per.nombre=\'CAMPAÑA 2023\' and prop.estado=\'activo\'')[0]['cantidad_props']
 

In [5]:
cantidad_usuarios

1388

In [6]:
cantidad_props

156

,name,apellido_paterno,apellido_materno,username,codigo
0,12504 - LAUREANA MARTINEZ,MARTINEZ,DE SALO,12504,12504
1,2641 - ROBERTO CRONENBOLD,CRONENBOLD,AGUILERA,rcronenbold,2641
2,2816 - ALBERTO DONOSO,DONOSO,BLANCO,adonoso,2816


In [10]:
lista_users.to_excel('lista_usuarios.xlsx')

# ALERTAS

In [18]:
alertas = dbQuery('select * from alerta where created_at > \'2023-03-01\' and comentario <> \'NAX\'')

In [19]:
lista_alertas = pd.DataFrame(alertas)

In [21]:
lista_alertas.to_excel('alertas.xlsx')

## ACTUALIZAR NOMBRES DE LABORES EN MI PRECIO

In [7]:
# al modificar los nombres de las actividades desde el admin, 
# los usuarios que ya tengan actividades con precios personalizados no se actualizan los nombres
# este script actualiza los nombres de actividades 'mi_precio' con los nombres se actividades globales
query = 'update tipo_actividad_mi_precio mi set mi.nombre=(select nombre from tipo_actividad ac where ac.id=mi.tipo_actividad_id)'
res = dbQuery(query)

In [26]:
res

()

In [9]:
# modifica los nombres de la lista de productos con precio personalizado 
# esto en caso de que el nombre del producto estandar cambie de nombre
query = 'update producto_act_miprecio pam set pam.nombre=(select p_a.nombre from producto_actividad p_a where p_a.id=pam.producto_actividad_id)'
res = dbQuery(query)